In [1]:
# Importações necessárias
import pandas as pd
import re
import nltk
import string 
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from ipywidgets import interact, widgets
from IPython.display import display, clear_output
import datetime as dt
import spacy
#from bertopic import BERTopic
#from sklearn.feature_extraction.text import CountVectorizer
#from groq import Groq
#from reportlab.lib.pagesizes import letter, A4
#from reportlab.pdfgen import canvas
import datetime
import os
#from textwrap import wrap

In [2]:
nltk.download('stopwords')
stop_words_pt = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\familia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Carregar o modelo em português
nlp = spacy.load("pt_core_news_sm")

In [4]:
# Carregar o DataFrame a partir do arquivo Excel
df_final = pd.read_excel('arquivogeral.xlsx')

In [5]:
# 1. Gráfico interativo de quantidade de mensagens por data
def grafico_mensagens_por_data(df):
    mensagens_por_data = df.groupby(df['data'].dt.date).size()
    mensagens_por_data_df = mensagens_por_data.reset_index(name='Quantidade de Mensagens')
    mensagens_por_data_df['data'] = mensagens_por_data_df['data'].apply(lambda x: x.strftime('%d-%m-%Y'))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mensagens_por_data_df['data'],
                             y=mensagens_por_data_df['Quantidade de Mensagens'],
                             mode='lines',
                             name='Quantidade de Mensagens'))

    fig.update_layout(
        xaxis=dict(
            title='Data (Dia-Mês-Ano)',
            range=[mensagens_por_data_df['data'].iloc[0], mensagens_por_data_df['data'].iloc[29]],
            rangeslider=dict(
                visible=True,
                range=[mensagens_por_data_df['data'].iloc[0], mensagens_por_data_df['data'].iloc[-1]]
            ),
            tickformat='%d-%m-%Y',
        ),
        yaxis=dict(title='Quantidade de Mensagens'),
        title='Quantidade de Mensagens por Data',
        dragmode='zoom',
        hovermode='x unified'
    )
    fig.show()

In [6]:
grafico_mensagens_por_data(df_final)

In [7]:
# 2. Gráfico interativo de quantidade de mensagens por autor e data
def grafico_mensagens_por_autor(df, autores_especificos):
    mensagens_por_autor_data_df = df.groupby(['data', 'username']).size().reset_index(name='Quantidade de Mensagens')
    filtro_df = mensagens_por_autor_data_df[mensagens_por_autor_data_df['username'].isin(autores_especificos)]

    fig1 = px.line(filtro_df,
                   x='data',
                   y='Quantidade de Mensagens',
                   color='username',
                   labels={"Data": "Data (Dia-Mês-Ano)", "Quantidade de Mensagens": "Quantidade de Mensagens"},
                   title='Quantidade de Mensagens por Autor e Data')

    fig1.update_layout(
        xaxis=dict(
            title='Data (Dia-Mês-Ano)',
            range=[filtro_df['data'].iloc[0], filtro_df['data'].iloc[29]],
            rangeslider=dict(
                visible=True,
                range=[filtro_df['data'].iloc[0], filtro_df['data'].iloc[-1]]
            ),
            tickformat='%d-%m-%Y',
        ),
        yaxis=dict(title='Quantidade de Mensagens'),
        dragmode='zoom',
        hovermode='x unified'
    )
    fig1.show()

In [8]:
grafico_mensagens_por_autor(df_final, ['jairbolsonarobrasil', 'LulanoTelegram'])

In [9]:
# 3. Gráficos de barras para emojis por grupo
def criar_grafico_reacoes(df, username):
    df['emojis_list'] = df['emojis'].str.split(', ')
    df['counts_list'] = df['counts'].str.split(', ').apply(lambda x: list(map(int, x)))
    df_expanded = df.explode(['emojis_list', 'counts_list'])
    df_expanded = df_expanded.rename(columns={'emojis_list': 'emoji', 'counts_list': 'count'})
    
    df_user = df_expanded[df_expanded['username'] == username]
    df_grouped = df_user.groupby('emoji', as_index=False)['count'].sum()

    fig = px.bar(df_grouped,
                 x='emoji',
                 y='count',
                 labels={'emoji': 'Reação (Emoji)', 'count': 'Quantidade de Reações'},
                 title=f'Quantidade de Reações por Emoji - {username}')
    fig.show()


In [10]:
criar_grafico_reacoes(df_final, 'jairbolsonarobrasil')

In [11]:
criar_grafico_reacoes(df_final, 'LulanoTelegram')

In [12]:
# Filtrar os usernames de cada grupo
bolsonaro_usernames = ["JNA2021", "filipetrielli1", "lulajamais2", "ianmaldonado", "realpfigueiredo",
                       "allandossantos", "siteolavete", "DireitaChannel", "FlaviaFerronato", "FabianaBarroso",
                       "aazzibarreto", "isentaocast", "eduperez80", "ludmilalinsgrilo", "JulioVSchneider",
                       "conexaopoliticabr", "PHVox", "arthurweintraub", "diretoaosfatos", "opropriolavo",
                       "domesdras", "filgmartin", "vidadestra", "grupob38", "souzaslaughter"]

lula_usernames = ["jornalistaslivres", "esquerdaonline", "progressistass", "+VPXlha9FZOcxOTgx", "voulula13",
                  "socialismos", "memesesquerdistas", "esquerdasunidascanal", "Andrejanonestelegram",
                  "lulaverso", "arsenaldolula"]

neutros_usernames = ["avozdocerrado", "garoaclube", "bbcbrasil", "tramadora", "SputnikBrasil"]

In [13]:
# Criar DataFrames filtrados por grupo
df_bolsonaro = df_final[df_final['username'].isin(bolsonaro_usernames)]
df_lula = df_final[df_final['username'].isin(lula_usernames)]
df_neutros = df_final[df_final['username'].isin(neutros_usernames)]

In [14]:
# Agrupar por data e contar a quantidade de mensagens
bolsonaro_por_data = df_bolsonaro.groupby('data').size().reset_index(name='Quantidade de Mensagens')
lula_por_data = df_lula.groupby('data').size().reset_index(name='Quantidade de Mensagens')
neutros_por_data = df_neutros.groupby('data').size().reset_index(name='Quantidade de Mensagens')

In [15]:
# Adicionar coluna para identificar o grupo
bolsonaro_por_data['Grupo'] = 'Bolsonaro'
lula_por_data['Grupo'] = 'Lula'
neutros_por_data['Grupo'] = 'Neutros'

In [16]:
# Concatenar os DataFrames
df_mensagens_por_data = pd.concat([bolsonaro_por_data, lula_por_data, neutros_por_data])

In [17]:
# Criar o gráfico de linhas
fig2 = px.line(df_mensagens_por_data,
              x='data',
              y='Quantidade de Mensagens',
              color='Grupo',
              labels={
                  "data": "Data",
                  "Quantidade de Mensagens": "Quantidade de Mensagens"
              },
              title='Quantidade de Mensagens por Data (Perfis Não Institucionais)')

# Ajustar a exibição inicial e adicionar funcionalidades de zoom e rolagem
fig2.update_layout(
    xaxis=dict(
        title='Data (Dia-Mês-Ano)',
        rangeslider=dict(
            visible=True,
            range=[df_mensagens_por_data['data'].min(), df_mensagens_por_data['data'].max()]  # Habilitar rolagem para o restante do período
        ),
        tickformat='%d-%m-%Y',  # Formato da data
    ),
    yaxis=dict(
        title='Quantidade de Mensagens'
    ),
    dragmode='zoom',  # Habilitar o modo de zoom
    hovermode='x unified'  # Exibir a informação da quantidade de mensagens ao passar o mouse
)

# Exibir o gráfico
fig2.show()

In [18]:
# Gráfico de Reações por Emoji para Perfis Não Institucionais

# Função para criar gráficos de barras de reações para cada grupo
def criar_grafico_reacoes_grupo(df, grupo_usernames, grupo_nome):
    df_grupo = df[df['username'].isin(grupo_usernames)]
    df_grouped = df_grupo.groupby('emoji', as_index=False)['count'].sum()

    fig = px.bar(df_grouped,
                 x='emoji',
                 y='count',
                 labels={
                     'emoji': 'Reação (Emoji)',
                     'count': 'Quantidade de Reações'
                 },
                 title=f'Quantidade de Reações por Emoji - {grupo_nome}')
    return fig

In [19]:
# Separar os emojis e as contagens em listas
df_final['emojis_list'] = df_final['emojis'].str.split(', ')
df_final['counts_list'] = df_final['counts'].str.split(', ').apply(lambda x: list(map(int, x)))


In [20]:
# Expandir as listas em várias linhas
df_expanded = df_final.explode(['emojis_list', 'counts_list'])
# Renomear as colunas expandidas
df_expanded = df_expanded.rename(columns={'emojis_list': 'emoji', 'counts_list': 'count'})

In [21]:
# Criar gráficos para os grupos
fig_reacoes_bolsonaro = criar_grafico_reacoes_grupo(df_expanded, bolsonaro_usernames, 'Bolsonaro (Perfis Não Institucionais)')
fig_reacoes_bolsonaro.show()

In [22]:
fig_reacoes_lula = criar_grafico_reacoes_grupo(df_expanded, lula_usernames, 'Lula (Perfis Não Institucionais)')
fig_reacoes_lula.show()


In [23]:
fig_reacoes_neutros = criar_grafico_reacoes_grupo(df_expanded, neutros_usernames, 'Neutros (Perfis Não Institucionais)')
fig_reacoes_neutros.show()

In [24]:
def limpar_mensagem(mensagem):
    if isinstance(mensagem, str):  # Verifica se a mensagem é uma string
        # Converter para minúsculas
        mensagem = mensagem.lower()

        # Remover números
        mensagem = re.sub(r'\d+', '', mensagem)

        # Remover URLs completamente
        mensagem = re.sub(r'http\S+|www\S+|https\S+', '', mensagem, flags=re.MULTILINE)

        # Remover espaços extras
        mensagem = mensagem.strip()
        mensagem = re.sub(r'\s+', ' ', mensagem)

        # Remover stop words
        palavras = mensagem.split()
        mensagem = ' '.join([palavra for palavra in palavras if palavra not in stop_words_pt])

        return mensagem
    else:
        return ''  # Retorna uma string vazia se não for uma string

In [25]:
# Aplicar a função de limpeza na coluna de conteúdo
df_final['Conteúdo Limpo'] = df_final['conteudo'].apply(limpar_mensagem)

In [26]:
# Agrupar o conteúdo por data e concatenar o conteúdo limpo
conteudo_por_data = df_final.groupby('data')['Conteúdo Limpo'].apply(' '.join).reset_index()

In [27]:
# Remover registros sem conteúdo e agrupar por dia
conteudo_com_texto = conteudo_por_data[conteudo_por_data['Conteúdo Limpo'].str.strip() != '']
conteudo_com_texto['Dia'] = conteudo_com_texto['data'].dt.date
conteudo_agrupado = conteudo_com_texto.groupby('Dia')['Conteúdo Limpo'].apply(' '.join).reset_index()

In [28]:
# Função para gerar nuvem de palavras
def gerar_nuvem_palavras(texto, titulo="Nuvem de Palavras"):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(titulo, fontsize=16)
    plt.show()

In [29]:
# Criar DataFrames para cada grupo específico
df_jairbolsonarobrasil = df_final[df_final['username'] == 'jairbolsonarobrasil']
df_lulanoTelegram = df_final[df_final['username'] == 'LulanoTelegram']
df_bolsonaro_usernames = df_final[df_final['username'].isin(bolsonaro_usernames)]
df_lula_usernames = df_final[df_final['username'].isin(lula_usernames)]
df_neutros_usernames = df_final[df_final['username'].isin(neutros_usernames)]

In [30]:
# Filtrar e agrupar o conteúdo limpo por data para cada grupo
conteudo_jairbolsonarobrasil_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_jairbolsonarobrasil['data'].dt.date)]
conteudo_lulanoTelegram_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_lulanoTelegram['data'].dt.date)]
conteudo_bolsonaro_usernames_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_bolsonaro_usernames['data'].dt.date)]
conteudo_lula_usernames_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_lula_usernames['data'].dt.date)]
conteudo_neutros_usernames_agrupado = conteudo_agrupado[conteudo_agrupado['Dia'].isin(df_neutros_usernames['data'].dt.date)]


In [31]:
# Função para gerar nuvem de palavras
def gerar_nuvem_palavras(texto):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [32]:
# Converter a coluna 'Dia' para datetime, se necessário
if not pd.api.types.is_datetime64_any_dtype(conteudo_agrupado['Dia']):
    conteudo_agrupado['Dia'] = pd.to_datetime(conteudo_agrupado['Dia'])

In [33]:
# Criar um dropdown interativo com todas as datas disponíveis
datas_disponiveis = conteudo_agrupado['Dia'].dt.strftime('%d-%m-%Y').unique()
dropdown = widgets.Dropdown(
    options=datas_disponiveis,
    description='data:',
    disabled=False,
)

In [34]:
# Função para mostrar a nuvem de palavras para o dia selecionado em diferentes grupos
def mostrar_nuvem_por_grupo(data_selecionada, grupo):
    data_formato = pd.to_datetime(data_selecionada, format='%d-%m-%Y').date()

    if grupo == 'jairbolsonarobrasil':
        filtro_data = conteudo_jairbolsonarobrasil_agrupado[conteudo_jairbolsonarobrasil_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'lulanoTelegram':
        filtro_data = conteudo_lulanoTelegram_agrupado[conteudo_lulanoTelegram_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'bolsonaro_usernames':
        filtro_data = conteudo_bolsonaro_usernames_agrupado[conteudo_bolsonaro_usernames_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'lula_usernames':
        filtro_data = conteudo_lula_usernames_agrupado[conteudo_lula_usernames_agrupado['Dia'] == data_formato]['Conteúdo Limpo']
    elif grupo == 'neutros_usernames':
        filtro_data = conteudo_neutros_usernames_agrupado[conteudo_neutros_usernames_agrupado['Dia'] == data_formato]['Conteúdo Limpo']

    if not filtro_data.empty:
        texto = filtro_data.values[0]
        gerar_nuvem_palavras(texto)
    else:
        print(f"Nenhum conteúdo encontrado para a data: {data_selecionada}")

In [35]:
# Conectar a função de exibição da nuvem de palavras ao dropdown
interact(mostrar_nuvem_por_grupo, data_selecionada=dropdown, grupo=['jairbolsonarobrasil', 'lulanoTelegram', 'bolsonaro_usernames', 'lula_usernames', 'neutros_usernames'])

interactive(children=(Dropdown(description='data:', options=('17-12-2015', '18-12-2015', '19-12-2015', '21-12-…

<function __main__.mostrar_nuvem_por_grupo(data_selecionada, grupo)>

In [36]:
# Criar DataFrames filtrados por grupo
df_bolsonaro = df_final[df_final['username'].isin(bolsonaro_usernames)]
df_lula = df_final[df_final['username'].isin(lula_usernames)]
df_neutros = df_final[df_final['username'].isin(neutros_usernames)]

In [37]:
# Função para lematizar o texto
#def lemmatize_text(text):
    #doc = nlp(text)
    #return " ".join([token.lemma_ for token in doc if not token.is_stop])

In [38]:
# Supondo que você tenha uma coluna 'mensagem' com os textos
#df_bolsonaro['mensagem_lemmatizada'] = df_bolsonaro['Conteúdo Limpo'].apply(lemmatize_text)
#df_lula['mensagem_lemmatizada'] = df_lula['Conteúdo Limpo'].apply(lemmatize_text)
#df_neutros['mensagem_lemmatizada'] = df_neutros['Conteúdo Limpo'].apply(lemmatize_text)

In [39]:
# Extrair as mensagens lematizadas
#documents = df_bolsonaro['mensagem_lemmatizada'].tolist()
#documents1= df_lula['mensagem_lemmatizada'].tolist()
#documents2= df_neutros['mensagem_lemmatizada'].tolist()

In [40]:
# Customizar o CountVectorizer
#vectorizer_model = CountVectorizer(
    #stop_words=stop_words_pt,  # Usar stopwords em português do NLTK
    #ngram_range=(1, 2),  # Considerar uni e bigramas
    #min_df=5,  # Ignorar termos que aparecem em menos de 5 documentos
    #max_df=0.95  # Ignorar termos que aparecem em mais de 95% dos documentos
#)


In [41]:
# Instanciar o modelo BERTopic com o vetor customizado
#topic_model = BERTopic(vectorizer_model=vectorizer_model)

In [42]:
#topics, _ = topic_model.fit_transform(documents)

In [43]:
# Atribuir os tópicos às mensagens Bolsonaristas
#df_bolsonaro['topic'] = topics

In [44]:
#num_unique_topics = df_bolsonaro['topic'].nunique()
#print(f"Quantidade de tópicos diferentes: {num_unique_topics}")

Quantidade de tópicos diferentes: 766


In [45]:
#topics1, _ = topic_model.fit_transform(documents1)

In [46]:
# Atribuir os tópicos às mensagens Lula
#df_lula['topic'] = topics1

In [47]:
#num_unique_topics1 = df_lula['topic'].nunique()
#print(f"Quantidade de tópicos diferentes: {num_unique_topics1}")

Quantidade de tópicos diferentes: 277


In [48]:
#topics2, _ = topic_model.fit_transform(documents2)

In [51]:
# Atribuir os tópicos às mensagens Neutros
#df_neutros['topic'] = topics2

In [52]:
#num_unique_topics2 = df_neutros['topic'].nunique()
#print(f"Quantidade de tópicos diferentes: {num_unique_topics2}")

Quantidade de tópicos diferentes: 46


In [53]:
#client = Groq(
    #api_key="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"
#)

In [54]:
#resumos_por_data = {}  # Dicionário global para armazenar resumos por data

#def visualizar_resumo_por_topico(df_selecionado, topico, limpar_resumos=True):
 #   global resumos_por_data  # Usar a variável global

  #  if limpar_resumos:
        #resumos_por_data = {}  # Limpar resumos anteriores

    # Filtrar o dataframe pelo tópico selecionado
   # df_topico = df_selecionado[df_selecionado['topic'] == topico]

    # Agrupar as mensagens por data
    #grupo_por_data = df_topico.groupby('data')

    #print(f"Resumo do Tópico {topico} agrupado por data (baseado nas 5 primeiras mensagens por data):")

    # Iterar pelas datas e gerar um resumo das 5 primeiras mensagens de cada uma
    #for data, grupo in grupo_por_data:
     #   mensagens = grupo['conteudo'].head(5).dropna().tolist()  # Remove NaNs
      #  mensagens = [str(mensagem) for mensagem in mensagens if isinstance(mensagem, str)]  # Filtra apenas strings
#
 ##          mensagens_texto = "\n".join(mensagens)  # Concatena as mensagens para enviar ao modelo
            
            # Enviar as mensagens ao modelo Llama para obter um resumo
   #         chat_completion = client.chat.completions.create(
    #            messages=[
     #               {
      #                  "role": "user",
       #                 "content": f"Abaixo estão algumas mensagens de {data} sobre o Tópico {topico}. Por favor, resuma o principal assunto discutido nessas mensagens:\n\n{mensagens_texto}"
        #            }
         #       ],
          #      model="llama3-70b-8192"
           # )
            
            ## Extrair o conteúdo da resposta do modelo
            #resumo = chat_completion.choices[0].message.content
            #resumos_por_data[data] = resumo  # Armazenar o resumo no dicionário
            
            #print(f"\nData: {data}")
            #print(f"Resumo: {resumo}")
        #else:
            #print(f"\nData: {data} - Sem mensagens disponíveis para resumo.")

    #return resumos_por_data

In [ ]:
# Função para salvar o resumo em PDF na pasta especificada
#def salvar_resumo_pdf1(resumos, topico):
    # Define a pasta onde os arquivos PDF serão salvos
 #   pasta_destino = "resumos_pdf"
  #  os.makedirs(pasta_destino, exist_ok=True)  # Cria a pasta se ela não existir

    # Define o nome do arquivo PDF com base no tópico
   # nome_arquivo = os.path.join(pasta_destino, f"resumo_topico_{topico}.pdf")
    
    # Configura o PDF com tamanho A4
    ##c = canvas.Canvas(nome_arquivo, pagesize=A4)
    #largura, altura = A4

    # Define a posição inicial para o texto
    #x, y = 50, altura - 50
    #c.setFont("Helvetica", 10)

    # Adiciona título no PDF
    #c.drawString(x, y, f"Resumo do Tópico {topico}")
    #y -= 30

    # Define a largura máxima da linha em caracteres
    #largura_maxima_linha = 80  # Ajuste conforme necessário para o layout desejado

    # Itera pelas datas e resumos para salvá-los no PDF
    #for data, resumo in resumos.items():
        # Adiciona a data ao PDF
     #   c.drawString(x, y, f"Data: {data}")
      #  y -= 15
        
        # Quebra o texto do resumo em várias linhas, se necessário
       # linhas_resumo = wrap(resumo, largura_maxima_linha)
        #for linha in linhas_resumo:
         #   if y < 40:  # Verifica se há espaço suficiente, caso contrário, cria nova página
          #      c.showPage()
           #     c.setFont("Helvetica", 10)
            #    y = altura - 50
            #c.drawString(x, y, linha)
            #y -= 15  # Move para a linha seguinte

        #y -= 10  # Adiciona um espaço entre resumos

    # Salva o PDF
    #c.save()
    #print(f"O PDF foi salvo em: {nome_arquivo}")

# Loop para gerar resumos para tópicos de 1 a 42 e salvar em PDF
#for topico in range(21, 44):
    #resumos = visualizar_resumo_por_topico(df_neutros, topico, limpar_resumos=True)
    #salvar_resumo_pdf1(resumos, topico)